# Assignment 6: Neural Networks
Josh Mau


## Overview
In this notebook, I have implemented a neural network running on data from the following source:
* http://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.cs

I have run two experiments on this neural network:
* Regression
* Classification

Each neural network is trained multiple times and the results are compared.

## References
The following python scripts were obtained at http://www.cs.colostate.edu/~anderson/cs440/notebooks/nn2.tar and were authored by Dr. Chuck Anderson
* neuralnetworks.py
* scaledconjugategradient.py
* mlutils.py

## Training the Neural Networks
This function takes inputs X and T where X is an input matrix of input data and T in an input matrix of target data. For the code below, the energydata_complete file from above was used to generate X and T. Additionally, trainNNs takes a trainFraction which splits the X and T input matricies into test data and train data. If 0.8 is passed as the value for trainFraction, 80% of the data will be used as training data and the remaining 20% will be used as test data. 

The hiddenLayerStructures is assumed a list containing lists that are hidden layer structures. This code block will be executed once for each item in this list. The neural network will be trained and tested using each of the hidden layer structure's passed. For instance, if the network structure array passed is:
<center>```[[1], [20, 20], [100, 50, 2, 50, 100]]```</center><br />
The code block will be executed 3 times. The first time using one hidden layer with one with one unit, the second will be 2 hidden layers with 20 units each, and the third will be 5 hidden layers, two with 100 units, two with 50 units, and one with 2 units.

Each neural network is trained ```numberRepetitions``` times. For each training session, the RMSE or root mean square error is calculated for both training and testing data and stored in a test or train array `numberRepetitions` in size.

trainNNs returns a list containing four items:
* The hidden layer structure (as a list)
* A list of training data performance for each repetition (list containing RMSE of training data)
* A list of testing data performance for each repetition (list containing RMSE of testing data)
* The number of seconds it took to run this many repetitions for this network structure.

For each hidden layer structure passed in the `hiddenLayerStructures` argument


In [ ]:
import sys
import time
import mlutils as ml
import numpy as np
import neuralnetworks as nn

def trainNNs(X, T, trainFraction, hiddenLayerStructures, numberRepetitions, numberIterations, classify=False):
    results = []    
    for hiddenLayer in hiddenLayerStructures:
        res = []
        res.append(hiddenLayer)
        rmseTest = []
        rmseTrain = []

        start = time.time()
        for n in range(numberRepetitions):
            Xtrain,Ttrain,Xtest,Ttest = ml.partition(X, T,  (trainFraction, 1-trainFraction), classify)
            # Neural Net takes Num Input, Num hidden Units, Num output
            hlSize = 1 if isinstance(hiddenLayer, int) else len(hiddenLayer)
            nnet = nn.NeuralNetwork(Xtrain.shape[1], hlSize, Ttrain.shape[1])
            nnet.train(Xtrain, Ttrain, numberIterations)

            if classify == False:
                rmseTest.append(round(np.sqrt(np.mean(nnet.use(Xtest)-Ttest)**2), 2))
                rmseTrain.append(round(np.sqrt(np.mean((nnet.use(Xtrain)-Ttrain)**2)), 2))
            else:
                rmseTest.append(np.sum(nnet.use(Xtest) != Ttest) / Ttest.shape[0])
                rmseTrain.append(np.sum(nnet.use(Xtrain) != Ttrain) / Ttrain.shape[0])
                
        end = time.time() - start
        end = round(end, 2)
        res.append(rmseTrain)
        res.append(rmseTest)
        res.append(end)
        results.append(res)
    
    return results

## Loading data file
This function reading in data in the file name passed. This function handles double quotes by removing them entirely, and splits each line on `,` (hence csv...) and stores each line in a list. The first and last two columns of this data file are not used thus the slicing of ```[1:-2]``` in the code below. The loadFrogs function returns Xanuran, Tanuran similar to X and T mentioned above, but also returns a list of species names. If a data item is classified as 0, the species will be `names[0]`

In [ ]:
def loadDataFile(filename):
    names = []
    data = []
    with open(filename, 'r') as f:
        names = f.readline().replace("\"", "").split(",")
        for line in f:
            temp = line.replace("\"", "").split(",")
            temp = [float(item.strip()) for item in temp[1:-2]]
            data.append(temp)

    dataNP = np.array(data)
    return names[1:-2], dataNP

def loadFrogs(filename):
    Xanuran = [] 
    Tanuran = []
    names = []
    with open(filename, 'r') as f:
        f.readline() # ignore first line
        for line in f:
            temp = line.split(",")
            tempX = temp[1:22]
            counter = 0
            for i in tempX:
                tempX[counter] = float(i)
                counter += 1
            Xanuran.append(tempX)
            if temp[24] not in names:
                names.append(temp[24])
            Tanuran.append([names.index(temp[24])])
    return np.array(Xanuran), np.array(Tanuran), names

## Summarize
This method takes in the results as discussed in the Training Neural Networks cell above. For each item in the parameter `results`, this function will compute the mean of each item in the training data performance, testing data performance, and time. The first hidden layer structure is returned but can be ignored as it has no statistical value.

In [227]:
def summarize(results):
    res = []
    for item in results:
        temp = []
        temp.append(item[0])
        trainMean = 0
        testMean = 0
        for train in item[1]:
            trainMean += train
        trainMean /= len(item[1])
        temp.append(trainMean)
        for test in item[2]:
            testMean += test
        testMean /= len(item[2])
        temp.append(testMean)
        temp.append(item[3])
        res.append(temp)
    return res

def bestNetwork(summary):
    min = float('inf')
    counter = 0
    min_index = 0
    for item in summary:
        if item[2] < min:
            min = item[2]
            min_index = counter
        counter += 1
    return summary[min_index]

## Regression Experiment
The neural network will be trained and tested without classification (using regression)

In [207]:
X = np.arange(10).reshape((-1,1))
T = X + 1 + np.random.uniform(-1, 1, ((10,1)))

# Train neural network
result = trainNNs(X, T, 0.8, [0, 1, 2, 10, [10, 10], [5, 5, 5, 5], [2]*5, [100, 50, 2, 50, 100], [16, 16], [1, 2, 3]], 50, 400, classify=False)
summ = summarize(result)
print("Best: ", end="")
print(bestNetwork(summ))

Best: [1, 0.44, 0.45000000000000001, 2.46]


### Discussion
The first item in the above is the best hidden layer structure, the second item is the training data error, and the third item is the testing data error percentage.

In [228]:
# Read in data file
names, data = loadDataFile('energydata_complete.csv')

# Store X and T
Xenergy = np.array([item[2:] for item in data])
Tenergy = np.array([item[:2] for item in data])

# Names
Xnames = np.array(names[2:])
Tnames = np.array(names[:2])

print("Xenergy shape: ", end="")
print(Xenergy.shape)
print("Tenergy shape: ", end="")
print(Tenergy.shape)

# Train neural network
result = trainNNs(Xenergy, Tenergy, 0.8, [0, 5, [5, 5], [10, 10]], 10, 100)
summ = summarize(result)
print(summ)
print("Best: ", end="")
print(bestNetwork(summ))

Xenergy shape: (19735, 24)
Tenergy shape: (19735, 2)
[[0, 69.974999999999994, 0.53200000000000003, 10.07], [5, 70.441999999999993, 0.75700000000000001, 10.37], [[5, 5], 66.543000000000006, 0.65199999999999991, 16.85], [[10, 10], 66.193000000000012, 0.96799999999999997, 17.22]]
Best: [0, 69.974999999999994, 0.53200000000000003, 10.07]


## Classification Experiment
The neural network will be trained and testing using classification

In [192]:
Xanuran, Tanuran, names = loadFrogs('Frogs_MFCCs.csv')
results = trainNNs(Xanuran, Tanuran, 0.8, [0, 5, [5, 5]], 5, 100, classify=True)
print(summarize(results))

[[0, 1.0, 1.0, 1.07], [5, 1.0, 1.0, 1.02], [[5, 5], 1.0, 1.0, 1.61]]


In [232]:
%run -i "A6grader.py"


Testing summarize([[[1,1], [1.2, 1.3, 1.4], [2.2, 2.3, 2.4], 0.5], [[2,2,2], [4.4, 4.3, 4.2], [6.5, 6.4, 6.3], 0.6]])

--- 10/10 points. Correctly returned [[[1, 1], 1.3, 2.3000000000000003, 0.5], [[2, 2, 2], 4.3, 6.3999999999999995, 0.6]]

Testing bestNetwork([[[1, 1], 1.3, 2.3, 0.5], [[2, 2, 2], 4.3, 1.3, 0.6]])

--- 10/10 points. Correctly returned [[2, 2, 2], 4.3, 1.3, 0.6]

X = np.random.uniform(-1, 1, (100, 3))
T = np.hstack(((X**2 - 0.2*X**3).sum(axis=1,keepdims=True),
               (np.sin(X)).sum(axis=1,keepdims=True)))
result = trainNNs(X, T, 0.7, [0, 5, 10, [20, 20]], 10, 100, False)

--- 20/20 points. Correct.

Testing bestNetwork(summarize(result))

--- 20/20 points. You correctly found that network [20, 20] is best.

A6 Execution Grade is 60/60

======================= The regression data set =======================

--- _/5 points. Read the data in energydata_complete.csv into variables Xenergy and Tenergy.

--- _/5 points. Train some networks by calling the NeuralNetw